### Vorhersage des GameStop Aktienkurses mit einem Rekurrenten neuronalen Netz

In diesem Notebook ist die Vorhersage
des Gamestop-Kurses mit einem rekurrenten Netz
in Python dokumentiert.

#### Rekurrente neuronale Netze

Rekurrente neuronale Netze beschreiben eine Familie von neuronalen Netzen
 für das Verarbeiten sequentieller Daten. Sie sind beispielsweise geeignet
 für die Vorhersage von Textsequenzen oder Zeitreihen. (vgl. Ian Goodfellow et al, Deep Learning, S. 365f)

Rekurrente neuronale Netze haben in der Regel je Zeitschritt ein hidden layer,
welches als Input das hidden layer des vorherigen Zeitschrittes bekommt, und als
Output für das nächste hidden layer dient:

![alt text](https://i.stack.imgur.com/BAgBq.jpg "RNN")
(Quelle: https://stats.stackexchange.com/questions/392112/tool-to-draw-neural-network-directed-graphs)
#### Architektur

Die Familie der rekurrenten Netze bietet eine Fülle von Architekturen für die
Zellen. In der Praxis haben sich insbesondere Architekturen mit "gated-cells" bewährt. Wodurch die Zellen mehr "Spielraum"
beim Lernen haben, indem sie beispielsweise durch ein Forget-Gate entscheiden, welche Informationen Sie sich merken oder
vergessen wollen.

Wir haben uns für eine Long short-term
memory Architektur entschieden, da sich diese im Allgemeinen und insbesondere im Aktienbereich bewährt hat
und auch in der Theorie eine gute Wahl darstellen sollte (vgl. Ian Goodfellow et al, Deep Learning, S. 397ff)

![alt text](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Long_Short_Term_Memory.png/400px-Long_Short_Term_Memory.png "LSTM")

(Grober Aufbau einer LSTM-Zelle, Quelle: https://de.wikipedia.org/wiki/Long_short-term_memory)

#### Vorhersage mit den Kursdaten als einziges Feature

Um den Einfluss der reddit Kommentare und Postings zu erkennen versuchen wir erst den Kurs nur mit der vorherigen
Kursentwicklung vorherzusagen, um dieses Ergebnis mit den späteren Ergebnissen zu vergleichen.

Die Daten sehen wie folgt aus:

In [6]:
import pandas as pd
df = pd.read_csv("../input/GME.csv")
print("Anzahl der Zeilen und Spalten:", df.shape)
df.head(5)

Anzahl der Zeilen und Spalten: (89, 7)


,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-01-04,19.000000,19.100000,17.15,17.250000,17.250000,10022500
1,2021-01-05,17.350000,18.080000,17.23,17.370001,17.370001,4961500
2,2021-01-06,17.340000,18.980000,17.33,18.360001,18.360001,6056200
3,2021-01-07,18.469999,19.450001,18.02,18.080000,18.080000,6129300
4,2021-01-08,18.180000,18.299999,17.08,17.690001,17.690001,6482000


Es handelt sich hierbei um die Daten vom 04.01.2021 - 12.05.2021, abrufbar unter finance.yahoo.com.
Wie schon im Word-Dokument spezifiziert, werden wir den "Close"-Wert vorhersagen, also den Kurs bei Börsenschluss des
jeweiligen Tages.

Als nächstes teilen wir die Daten in Training- und Test auf. Die ersten 70 Datensätze werden die
Trainingsdaten, die letzten 19 die Testdaten:

In [7]:
train_data = df.iloc[:70, 4:5].values
test_data = df.iloc[70:, 4:5].values

Um die Performance zu verbessern normalisieren wir die Daten mithilfe des Min-Max Scalers von Scikit-learn:

In [8]:
from sklearn.preprocessing import MinMaxScaler
normalizer = MinMaxScaler(feature_range=(0, 1))
train_data_normalized = normalizer.fit_transform(train_data)
test_data_normalized = normalizer.fit_transform(test_data)

Als nächstes werden die Trainingsdaten vorbereitet. Für ein LSTM benötigen wir einen
drei-dimensionalen Input, bestehend aus Sample, Time Steps und Features.
Ein Sample ist eine Zeitreihe, welche Features zu den jeweiligen Time Steps beinhaltet.

Wir probieren zunächst einen Input von 60 Samples mit 10 Time Steps mit einem Feature(Kurs zum Close) aus.

Ein Sample ist also eine Zeitreihe von 10 Kurswerten am jeweiligen Börsenschluss. Die erste Zeitreihe sind also die Kurse vom
04.01.2021 - 15.01.2021(Freitag). Die zweite Zeitreihe sind die Kurse vom 05.01.2021 - 19.01.2021(Dienstag) usw. - der Montag war
der Martin-Luther-King Day in den USA, weswegen die Börsen geschlossen waren.

Unten sieht man beispielhaft die Werte der beiden ersten Samples.

In [9]:
import numpy as np
X_train = []
y_train = []
for i in range(10, 70):
    X_train.append(train_data_normalized[i-10:i, 0])
    y_train.append(train_data_normalized[i,0])
X_train = np.array(X_train)
y_train = np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
print("Sample 1:\n", X_train[0])
print("Sample 2:\n", X_train[1])


Sample 1:
 [[0.        ]
 [0.00036335]
 [0.00336099]
 [0.00251317]
 [0.00133229]
 [0.0081451 ]
 [0.00817538]
 [0.04284503]
 [0.06861261]
 [0.05525949]]
Sample 2:
 [[0.00036335]
 [0.00336099]
 [0.00251317]
 [0.00133229]
 [0.0081451 ]
 [0.00817538]
 [0.04284503]
 [0.06861261]
 [0.05525949]
 [0.06694725]]


Jetzt können wir das Netz bauen. Zu Beginn bauen wir ein Modell mit zehn Neuronen und einem Hidden Layer. Es wird jedoch
noch mit weiteren Konfigurationen probiert, bessere Ergebnisse zu erzielen. Um Overfitting zu minimieren wird ein Dropout-Layer
ebenfalls hinzugefügt. Wir setzen return_sequences auf False, da das ziel ist, immer nur den nächsten Tag vorherzusagen.

In [10]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

model = Sequential()

# input layer
model.add(LSTM(units=10, return_sequences = True, input_shape = (10, 1)))
model.add(Dropout(0.2))

# hidden layer
model.add(LSTM(units=10, return_sequences = False))

# output layer
model.add(Dense(units = 1))

model.compile(optimizer = "adam", loss = "mean_absolute_percentage_error")
model.fit(X_train, y_train, epochs = 100, batch_size = 32)


Epoch 1/100
2/2 [==============================] - 0s 3ms/step - loss: 93.6631
Epoch 2/100
2/2 [==============================] - 0s 3ms/step - loss: 83.2078
Epoch 3/100
2/2 [==============================] - 0s 4ms/step - loss: 72.0267
Epoch 4/100
2/2 [==============================] - 0s 3ms/step - loss: 63.7060
Epoch 5/100
2/2 [==============================] - 0s 3ms/step - loss: 57.8418
Epoch 6/100
2/2 [==============================] - 0s 3ms/step - loss: 55.1345
Epoch 7/100
2/2 [==============================] - 0s 3ms/step - loss: 55.8430
Epoch 8/100
2/2 [========================